In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [3]:
data = spark.read.csv('SMSSpamCollection', inferSchema=True, sep='\t')

In [4]:
data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [5]:
data.describe().show()

+-------+----+--------------------+
|summary| _c0|                 _c1|
+-------+----+--------------------+
|  count|5574|                5574|
|   mean|null|               645.0|
| stddev|null|                 NaN|
|    min| ham| &lt;#&gt;  in mc...|
|    max|spam|… we r stayin her...|
+-------+----+--------------------+



In [6]:
data.show(truncate=False, n=2)

+---+---------------------------------------------------------------------------------------------------------------+
|_c0|_c1                                                                                                            |
+---+---------------------------------------------------------------------------------------------------------------+
|ham|Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...|
|ham|Ok lar... Joking wif u oni...                                                                                  |
+---+---------------------------------------------------------------------------------------------------------------+
only showing top 2 rows



In [7]:
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')
data.show(2, truncate=False)

+-----+---------------------------------------------------------------------------------------------------------------+
|class|text                                                                                                           |
+-----+---------------------------------------------------------------------------------------------------------------+
|ham  |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...|
|ham  |Ok lar... Joking wif u oni...                                                                                  |
+-----+---------------------------------------------------------------------------------------------------------------+
only showing top 2 rows



## Clean and Prepare the Data

** Create a new length feature: **

In [8]:
from pyspark.sql.functions import length

In [9]:
data = data.withColumn('length', length(data['text']))
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [10]:
# Pretty clear difference
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



# Feature Transformations

In [11]:
from pyspark.ml.feature import CountVectorizer,StringIndexer,Tokenizer,StopWordsRemover,IDF

In [12]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_txt')
stopremove = StopWordsRemover(inputCol='token_txt', outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_num = StringIndexer(inputCol='class', outputCol='label')

In [13]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [14]:
clean_up = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')

# Train the model
Using Naive Bayes

In [15]:
from pyspark.ml.classification import NaiveBayes

In [16]:
# Use Deafult
nb = NaiveBayes()

# Pipeline

In [17]:
from pyspark.ml import Pipeline

In [18]:
data_prep_line = Pipeline(stages=[ham_spam_to_num,tokenizer, stopremove, count_vec, idf, clean_up])

In [19]:
cleaner = data_prep_line.fit(data)
clean_data = cleaner.transform(data)
clean_data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)
 |-- label: double (nullable = true)
 |-- token_txt: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- stop_tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- c_vec: vector (nullable = true)
 |-- tf_idf: vector (nullable = true)
 |-- features: vector (nullable = true)



In [20]:
clean_data = clean_data.select(['label', 'features'])
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [21]:
train_data, test_data = clean_data.randomSplit([0.7, 0.3])
spam_predictor = nb.fit(train_data)

In [22]:
train_data.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [23]:
test_results = spam_predictor.transform(test_data)
test_results.show(2)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[-794.09787957731...|[1.0,1.3353547265...|       0.0|
|  0.0|(13424,[0,1,2,41,...|[-1058.7098726489...|[1.0,6.1906763694...|       0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 2 rows



In [24]:
test_results.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)



# Evaluation

In [25]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [26]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print('Accuracy of model at predicting spam was: {}'.format(acc))

Accuracy of model at predicting spam was: 0.9212918071091879


we coould imporve accuracy by switching out the classification models! Or try to come up with other engineered features!

### Try Logistic Regression Model

In [27]:
from pyspark.ml.classification import LogisticRegression

In [28]:
# Initialize Logistic Regression model
log_reg = LogisticRegression()
# Train the model
lg_model = log_reg.fit(train_data)

# get test results
test_results = lg_model.transform(test_data)
evaluator = MulticlassClassificationEvaluator()
lg_accuray = evaluator.evaluate(test_results)
print('Logistic Regression Accuracy: {}'.format(lg_accuray))

Logistic Regression Accuracy: 0.9754465930982877


It looks that logistic regression model performs much better than naive bayes